# Part I. ETL Pipeline for Pre-Processing the Files

in this Part of the program trought a python pipe all the data in the folder event_data was condensed in the CSV document event_datafile_new.csv 

the columns of event_datafile.csv are: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId


In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [3]:
#current working directory
print(os.getcwd())

#current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

#for loop with list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating file called event_datafile_full csv
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra (CQL code)

The next part of the program was designed to answer three questions with an Apache Cassandra database (no-relational). 

-1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

-2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

-3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [6]:
# connection to Cassandra instance your local machine (127.0.0.1)
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [7]:
#  created Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [8]:
# conected new Keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# Query 1 (artist_song_length_from_session)
The Question 1 expects Name of the artist, title of the song and length of the track based on sessionId and itemInSession.

1) The expected output is : "Name of the artist, title of the song and length of the track"
    
2) Based on (Primary key) : "sessionId and itemInSession"

In [9]:

query = "CREATE TABLE IF NOT EXISTS artist_song_length_from_session"
query = query + "(sessionId int, itemInSession int, artist text, song text, length double, \
PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
            
## INSERT statements into the `query` variable            
        query = "INSERT INTO artist_song_length_from_session (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        ## Insert data from event_datafile_new.csv in the NoSQL table artist_song_length_from_session
        try:
            session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        except Exception as e:
            print(e)

In [11]:
## verify the data was entered into the table with sessionId = 338 and itemInSession = 4

query = "select artist, song, length from artist_song_length_from_session WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)
#    print(row.artist, row.song, row.length)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


# Query 2 (song_playlist_sesion)
The Question 2 expects Name of the artist, title of the song and the first and last namen of the user based on userID and sessionId. The table was organized descresing orther of itemInSession

1) The expected output is : "Name of the artist, title of the song, first and last name of the user"
    
2) Based on (Primary Key) : "userId and sessionId" 

3) organized bei (clusterning column): "itemInSession"

In [12]:
query2 = "CREATE TABLE IF NOT EXISTS song_playlist_sesion"
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text,  PRIMARY KEY ((userId, sessionId),itemInSession))"
try:
    session.execute(query2)
#    print(query2)
except Exception as e:
    print(e)

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    i = 0
    for line in csvreader:
            
## INSERT statements into the `query` variable            
        query = "INSERT INTO song_playlist_sesion (artist, song, firstName, lastName, itemInSession, userId, sessionId)"
        query = query + " VALUES (%s, %s, %s, %s, %s ,%s ,%s)"
         ## Insert data from event_datafile_new.csv in the NoSQL table song_playlist_sesion
        try:
            session.execute(query, (line[0], line[9], line[1], line[4], int(line[3]), int(line[10]), int(line[8])))
        except Exception as e:
            print(e)

In [14]:
# verify the data was entered into the table with userId = 10 and sessionId = 182
query2 = "select artist, song, firstname, lastname from song_playlist_sesion WHERE userId = 10 and sessionId = 182"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


# Query 3 (username_song_listener)
The Question 3 expects first and last namen of the user track based on song and userId.

1) The expected output is : "Name of the artist, title of the song, first and last name of the user"
    
2) Based on (Primary Key) : "userId and sessionId" 

3) organized bei (clusterning column): "itemInSession"

In [15]:
query3 = "CREATE TABLE IF NOT EXISTS username_song_listener"
query3 = query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    i = 0
    for line in csvreader:
## INSERT statements into the `query` variable              
        query3 = "INSERT INTO username_song_listener (song, userId, firstName, lastName)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        ## Insert data from event_datafile_new.csv in the NoSQL table username_song_listener
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

In [17]:
# verify the data was entered into the table with the song All Hands Against His Own
query3 = "select firstName, lastName from username_song_listener WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query3)
    print(rows)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [19]:
query = "drop table artist_song_length_from_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_playlist_sesion"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table username_song_listener"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()